In [2]:
import bs4
import bs4.builder._lxml
import urllib
import pandas as pd
import numpy as np
import requests
import urllib.request, json 
import re
import time
from datetime import date

import hashlib

def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    return h.hexdigest()
    
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums, check_extractable=False):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text
    
def download_pdf(url,path,filename):
    try:
        
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

def connect_to_page(url):
    connection = requests.get(url)
    html = connection.text
    soup = bs4.BeautifulSoup(html,"lxml")
    connection.close()
    return soup
def download_pdf_unmasked(url,path,filename):
    
        
    filename = path+filename

    f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

    with open(filename, "wb") as code:
        #code.write(f.read())
        code.write(f.content)

    return convert(filename)

    
def download_docx(book_link, book_name):
    the_book = requests.get(book_link, stream=True)
    with open(book_name, 'wb') as f:
      for chunk in the_book.iter_content(1024 * 1024 * 2):  # 2 MB chunks
        f.write(chunk)

In [ ]:
BASE = "https://supremecourt.nebraska.gov"
source = "https://supremecourt.nebraska.gov/forms"
#cat 200 has 2 pages


soup = connect_to_page(source)

types = soup.find("select",id="edit-field-form-type-tid")
option = types.find_all("option")
cat_values = []
cat_names = []
for i in option:
    if i['value'].isdigit():
        cat_values.append(int(i['value']))
        cat_names.append(i.text)
print(cat_values)
print(cat_names)

#200 is 2 pages

In [13]:
today = date.today().strftime("%Y-%m-%d")
files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
form_dest = "PDFS/NE_forms/"
jur = 'NE'

In [ ]:
source = "https://supremecourt.nebraska.gov/forms"
second_page = "&page=1"

pdfs = {}


for index in range(len(cat_values)):
    start = "https://supremecourt.nebraska.gov/forms?title=&field_form_number_value=&field_form_type_tid={}&field_language_tid=All".format(cat_values[index])
    soup = connect_to_page(start)

    a = soup.find_all("a",href=True)

    for link in a:
        if ".pdf" in link['href']:
            filename = link['href'].split("/")[-1]
            
            
            if filename not in pdfs:
                title = re.sub(r"\([^()]*\)","",link.text)
                pdfs[filename] = title
                if title == 'here':
                    continue
                print(filename, link['href'])
                url_form = link['href']
                group = cat_names[index]
                id = hashme(url_form)
                download_pdf(url_form,form_dest,id+".pdf")
                files_df = files_df.append(pd.DataFrame([[id,jur,source,title,group,url_form,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))
                
    if cat_values[index] == 200:
        start = "https://supremecourt.nebraska.gov/forms?title=&field_form_number_value=&field_form_type_tid={}&field_language_tid=All".format(cat_values[index]) + second_page
        soup = connect_to_page(start)
        a = soup.find_all("a",href=True)

        for link in a:
            if ".pdf" in link['href']:
                filename = link['href'].split("/")[-1]
                
                
                if filename not in pdfs:
                    title = re.sub(r"\([^()]*\)","",link.text)
                    pdfs[filename] = title
                    if title == 'here':
                        continue
                    print(filename, link['href'])
                    url_form = link['href']
                    group = cat_names[index]
                    id = hashme(url_form)
                    status = download_pdf(url_form,form_dest,id+".pdf")
                    if status!='error':
                        files_df = files_df.append(pd.DataFrame([[id,jur,source,title,group,url_form,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]))
                    else:
                        print("############################")
                        print('ERROR')
                        print("############################")
files_df.to_csv("NE_metadata.csv",index=False, encoding="utf-8")